In [54]:
# Check json format errors
import json
import os

foler_list = ['gpt','davinci','gpt4']

for folder_path in foler_list:

    files = os.listdir(folder_path)

    def process_json_file(file_path):
        with open(file_path, 'r', encoding='utf-8') as input_file:
            line_number = 0
            for line in input_file:
                try:
                    data = json.loads(line)
                except json.JSONDecodeError:
                    print(f"{file_path}, line {line_number + 1}: JSON error")
                line_number += 1

    for file in files:
        full_path = os.path.join(folder_path, file)
        process_json_file(full_path)
        print(f"{full_path} completed")

gpt/few.jsonl completed
gpt/few_code_skel.jsonl completed
gpt/few_code.jsonl completed
davinci/zero.jsonl completed
davinci/few.jsonl completed
gpt4/zero_code_skel.jsonl completed
gpt4/zero_final.jsonl completed
gpt4/few.jsonl completed
gpt4/few_code.jsonl completed


In [79]:
import json  
import os

from nebula2.gclient.net import ConnectionPool
from nebula2.Config import Config
# from nebula2.data.ResultSet import ResultSe
# config
config = Config()
config.max_connection_pool_size = 10
# connect
connection_pool = ConnectionPool()
ok = connection_pool.init([('127.0.0.1', 9669)], config)
session = connection_pool.get_session('root', 'nebula')

foler_list = ['gpt','davinci','gpt4']

def prosess_db_test(graph_name,data):
    for line in data:
        use_graph = f'USE {graph_name}'
        session.execute(use_graph)
        content = line.get('gpt')
        result = line.get('result')
        gpt_result_list = []
        gpt_result = session.execute(content)
        if gpt_result.is_succeeded() == False:
            line['error'] = ["语法错误"]
        else:
            n = gpt_result.row_size()
            for i in range(n):
                gpt_result_list.append(str(gpt_result.row_values(i)))
            if len(gpt_result_list) == 0 and len(result) != 0:
                line['error'] = ["语法错误"]
            elif len(gpt_result_list) != 0 and gpt_result_list[0] == "[__NULL__]" and len(result) != 0:
                line['error'] = ["语法错误"]
        line['gpt_result'] = gpt_result_list
        output_result.append(line)

for folder_path in foler_list:

    output_path = 'output/' + folder_path
    
    if not os.path.exists(output_path):
        os.mkdir(output_path)

    files = os.listdir(folder_path)

    for file in files:

        full_path = os.path.join(folder_path, file)
        output_file = "output_" + file
        output_full_path = os.path.join(output_path, output_file)

        class_disease = []
        class_potter = []
        class_nba = []
        output_result = []

        with open(full_path, 'r') as input_file, open(output_full_path, 'w') as output_file:  
            for line in input_file:  
                data = json.loads(line)
                if data.get('class') == "disease":
                    class_disease.append(data)
                elif data.get('class') == "potter":
                    class_potter.append(data)
                elif data.get('class') == "nba":
                    class_nba.append(data)

            prosess_db_test('disease', class_disease)
            prosess_db_test('harrypotter_new', class_potter)
            prosess_db_test('nba', class_nba)

            json.dump(output_result, output_file, ensure_ascii=False, indent=4)
        print(output_full_path+" success")

[2023-10-31 16:00:23,458]:Get connection to ('127.0.0.1', 9669)


output/gpt/output_few.jsonl success
output/gpt/output_few_code_skel.jsonl success
output/gpt/output_few_code.jsonl success
output/davinci/output_zero.jsonl success
output/davinci/output_few.jsonl success
output/gpt4/output_zero_code_skel.jsonl success
output/gpt4/output_zero_final.jsonl success
output/gpt4/output_few.jsonl success
output/gpt4/output_few_code.jsonl success
